# Die Suche nach der optimalen Strategie

Beim Spiel **Würfel trifft** mit 3 Feldern und 6 Chips gibt es insgesamt 28 Strategien (Verteilungen der 6 Chips). Da kann noch jede Strategie gegen jede andere Strategie antreten. Die Anzahl an möglichen Strategien wächst aber schnell an. So gibt es z. B. beim Spiel **Differenz trifft** mit 6 Federn und 18 Chips 33649 Setzstrategien. Es ist hoffnungslos, jede gegen jede antreten zu lassen, um eine optimale Strategie zu finden. Eine Möglichkeit besteht darin, 
- sich auf eine Teilmenge aller Strategien zu beschränken.
- diese Strategien gegen Strategien antreten zu lassen, von denen man weiß, dass sie nur gegen wenige andere eine kleinere Gewinnwahrscheinlichkeit besitzen.

Dieses Vorgehen wird mit dem folgenden Programm verwirklicht.

Die (sinnvolle) Teilmenge wird durch 
- minFelder = (3, 4, 3, 2, 1, 0);   Mindestanzahl an Chips auf den Feldern 1 bis 6
- maxFelder = (5, 8, 7, 5, 2, 1); ; Maximalanzahl an Chips auf den Feldern 1 bis 6

realisiert. 

In diesem Beispiel müssen mindestens 3 und höchstens 5 Chips auf das 1. Feld gesetzt werden, u.s.w. 

In [2]:
from itertools import combinations_with_replacement

# Eingabe            
p = [6/36, 10/36, 8/36, 6/36, 4/36, 2/36]

minFelder = (3, 4, 3, 2, 1, 0)
maxFelder = (5, 8, 7, 5, 2, 1)

# Benchmark-Strategien, aussichsreiche Strategien
S = (3, 5, 4, 3, 2, 1)
T = (3, 6, 4, 3, 2, 0)

# Anzahl an Chips
total_chips = 18


def P(V, W, p, player='A'):
    state = (tuple(V), tuple(W), player)
    if state in memo:
        return memo[state]
    
    if all(v == 0 for v in V) and any(w > 0 for w in W):
        result = 1 if player == 'A' else 0
    elif all(w == 0 for w in W) and any(v > 0 for v in V):
        result = 1 if player == 'B' else 0
    elif all(v == w for v, w in zip(V, W)):
        result = 0
    else:
        total_prob = sum(p[j] for j in range(len(V)) if V[j] >= 1 or W[j] >= 1)
        result = 0

        for j in range(len(V)):
            if V[j] >= 1 and W[j] >= 1:
                V_new, W_new = list(V), list(W)
                V_new[j] -= 1
                W_new[j] -= 1
                result += p[j] / total_prob * P(V_new, W_new, p, player)
            elif W[j] >= 1 and V[j] == 0:
                W_new = list(W)
                W_new[j] -= 1
                result += p[j] / total_prob * P(V, W_new, p, player)
            elif V[j] >= 1 and W[j] == 0:
                V_new = list(V)
                V_new[j] -= 1
                result += p[j] / total_prob * P(V_new, W, p, player)
    
    memo[state] = result
    return result

def calculate_probabilities(U, V, p):
    P_A_win = P(U, V, p, 'A')
    P_B_win = P(U, V, p, 'B')
    P_draw = 1 - P_A_win - P_B_win
    return P_A_win, P_B_win, P_draw

def generate_strategies(total_chips, num_fields, minFelder, maxFelder):
    for c in combinations_with_replacement(range(num_fields), total_chips):
        strategy = [0] * num_fields
        for i in c:
            strategy[i] += 1
        
        if all(minFelder[i] <= strategy[i] <= maxFelder[i] for i in range(num_fields)):
            yield tuple(strategy)

# Eingabe            
p = [6/36, 10/36, 8/36, 6/36, 4/36, 2/36]

minFelder = (3, 4, 3, 2, 1, 0)
maxFelder = (5, 8, 7, 5, 2, 1)

# Benchmark-Strategien
S = (3, 5, 4, 3, 2, 1)
T = (3, 6, 4, 3, 2, 0)

# Anzahl an Chips
total_chips = 18

# Berechnungen
strategies = list(generate_strategies(total_chips, len(minFelder), minFelder, maxFelder))
print(f"Anzahl der Strategien nach Reduktion: {len(strategies)}")

best_strategies = {S,T}  # Menge der potenziell besten Strategien
for strategy in strategies:
    memo = {}
    P_vs_S, P_S_vs_strategy, _ = calculate_probabilities(strategy, S, p)
    P_vs_T, P_T_vs_strategy, _ = calculate_probabilities(strategy, T, p)
    
    if P_S_vs_strategy < P_vs_S or P_T_vs_strategy < P_vs_T:
        best_strategies.add(strategy)

print("Beste Strategien nach Vorselektion:")
for strat in best_strategies:
    print(strat)

Anzahl der Strategien nach Reduktion: 119
Beste Strategien nach Vorselektion:
(3, 6, 4, 3, 2, 0)
(3, 7, 4, 3, 1, 0)
(3, 5, 4, 3, 2, 1)
